In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# get data
path = '../data/external/MarathonData.csv'
pathNew = './MarathonData.csv'
df = pd.read_csv(pathNew)
# set id as index
df.set_index('id', inplace=True)
df

,Marathon,Name,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime,CATEGORY
id,,,,,,,,,
1,Prague17,Blair MORGAN,MAM,132.8,14.434783,NaN,1.16,2.37,A
2,Prague17,Robert Heczko,MAM,68.6,13.674419,NaN,1.23,2.59,A
3,Prague17,Michon Jerome,MAM,82.7,13.520436,NaN,1.30,2.66,A
4,Prague17,Daniel Or lek,M45,137.5,12.258544,NaN,1.32,2.68,A
5,Prague17,Luk ? Mr zek,MAM,84.6,13.945055,NaN,1.36,2.74,A
...,...,...,...,...,...,...,...,...,...
83,Prague17,Stefano Vegliani,M55,50.0,10.830325,NaN,2.02,3.93,D
84,Prague17,Andrej Madliak,M40,33.6,10.130653,ciclista 3h,1.94,3.93,D
85,Prague17,Yoi Ohsako,M40,55.4,11.043189,NaN,1.94,3.94,D


In [5]:
# convert to float all values
df = df.apply(pd.to_numeric, errors='coerce')

In [6]:
# if it does cross training set it to 1 else 0
df['cross_training'] = np.where(df['CrossTraining'].notnull(), 1, 0)
df

,Marathon,Name,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime,CATEGORY,cross_training
id,,,,,,,,,,
1,NaN,NaN,NaN,132.8,14.434783,NaN,1.16,2.37,NaN,0
2,NaN,NaN,NaN,68.6,13.674419,NaN,1.23,2.59,NaN,0
3,NaN,NaN,NaN,82.7,13.520436,NaN,1.30,2.66,NaN,0
4,NaN,NaN,NaN,137.5,12.258544,NaN,1.32,2.68,NaN,0
5,NaN,NaN,NaN,84.6,13.945055,NaN,1.36,2.74,NaN,0
...,...,...,...,...,...,...,...,...,...,...
83,NaN,NaN,NaN,50.0,10.830325,NaN,2.02,3.93,NaN,0
84,NaN,NaN,NaN,33.6,10.130653,NaN,1.94,3.93,NaN,0
85,NaN,NaN,NaN,55.4,11.043189,NaN,1.94,3.94,NaN,0


In [7]:
#  drop rows with wall21 null
df = df.dropna(subset=['Wall21'])

In [8]:
y = df.MarathonTime
X = df[['cross_training', 'km4week', 'sp4week']]

In [9]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
train_X, val_X, train_y, val_y = train_test_split(X, y)

In [14]:
# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],  # number of trees
    'max_depth': [10, 20, None],      # maximum depth of trees
    'min_samples_split': [2, 5, 10],  # minimum samples required to split
    'min_samples_leaf': [1, 2, 4],    # minimum samples required at leaf node
    'max_features': ['auto', 'sqrt']  # number of features to consider at each split
}


In [18]:
# Create base model
rf = RandomForestRegressor(random_state=1)

# Create GridSearchCV object
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                          # 5-fold cross-validation
    scoring='neg_mean_absolute_error',
    n_jobs=-1,                     # use all available cores
    verbose=2                      # print progress
)


In [19]:

# Fit the grid search
grid_search.fit(train_X, train_y)

# Print best parameters
print("Best parameters:", grid_search.best_params_)


Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=3

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
405 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
388 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py", line 1344, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.1

In [20]:

# Make predictions using the best model
best_model = grid_search.best_estimator_
pred = best_model.predict(val_X)
print("Mean Absolute Error:", mean_absolute_error(val_y, pred))

Mean Absolute Error: 0.1633643688586545


In [22]:
print(pred)

[3.2261 2.9363 2.8149 3.2376 3.547  3.5155 3.2261 2.7373 3.3091 3.6528
 3.12   3.4634 2.9674 3.5788 3.5356 3.6533 3.8387 3.4874 3.3877 3.2895
 3.4859]
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   

In [12]:
# save the model
from joblib import dump
dump(forest_model, '../models/marathon_model.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '../models/marathon_model.joblib'